In [1]:
import pickle 
import torch
import numpy as np 
import pandas as pd

In [2]:
from utils.config import get_config
from baselines.dnnperf import create_sample_graph
from scipy.sparse import csr_matrix
import dgl
from modules.latency_data import get_matrix_and_ops, get_adjacency_and_features
args = get_config()
graph, label = get_matrix_and_ops([0, 2, 1, 3, 4, 2])
graph, features = get_adjacency_and_features(graph, label)
graph = graph[1:, 1:]
graph = dgl.to_bidirected(dgl.from_scipy(csr_matrix(graph)))
key = torch.argmax(torch.tensor(features).long(), dim=1)[1:]
graph = create_sample_graph(key, graph, args)

Layer Type: input
Final Node Feature Vector h: tensor([3.0000e+00, 3.0000e+00, 1.2288e+04, 1.2288e+04, 0.0000e+00, 0.0000e+00,
        1.6000e+01])

Layer Type: ReLUConvBN
Final Node Feature Vector h: tensor([0.0000e+00, 3.0000e+00, 1.2288e+04, 6.5536e+04, 1.9200e+02, 9.8304e+04,
        1.6000e+01])

Layer Type: ReLUConvBN
Final Node Feature Vector h: tensor([0.0000e+00, 3.0000e+00, 1.2288e+04, 6.5536e+04, 1.9200e+02, 9.8304e+04,
        1.6000e+01])

Layer Type: ReLUConvBN
Final Node Feature Vector h: tensor([0.0000e+00, 3.0000e+00, 1.2288e+04, 6.5536e+04, 1.9200e+02, 9.8304e+04,
        1.6000e+01])

Layer Type: ReLUConvBN
Final Node Feature Vector h: tensor([1.0000e+00, 3.0000e+00, 1.2288e+04, 5.7600e+04, 1.7280e+03, 7.7760e+05,
        1.6000e+01])

Layer Type: ReLUConvBN
Final Node Feature Vector h: tensor([0.0000e+00, 3.0000e+00, 1.2288e+04, 6.5536e+04, 1.9200e+02, 9.8304e+04,
        1.6000e+01])

Layer Type: ReLUConvBN
Final Node Feature Vector h: tensor([0.0000e+00, 3.0000e+0

In [12]:
graph.ndata['h']

tensor([[0.7500, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000]])

In [ ]:
from utils.utils import makedir
import os
address = './results/ours/only_embed/log'
makedir('results')
# 打开输出文件
with open('./results/output.txt', 'w') as ans:
    # 获取目标目录中的所有文件
    all_result = os.listdir(address)
    
    # 遍历所有文件
    for filename in all_result:
        # 构建文件路径
        file_path = os.path.join(address, filename)
        
        # 打开并读取文件内容
        with open(file_path, 'r') as f:
            content = f.read()
            
            # 打印文件内容
            print(content)
            
            # 将文件内容写入输出文件，添加一些格式
            ans.write(f"Content of {filename}:\n")
            ans.write(content)
            ans.write("\n\n")

In [2]:
import pickle 

In [4]:
def find_closest_power_of_2(train_size, percentage=0.10):
    # 计算目标 batch size
    target_bs = int(train_size * percentage)
    
    # 定义 2 的幂次序列
    powers_of_2 = [2**i for i in range(1, 9)]  # 2, 4, 8, ..., 256

    # 找到最接近的 2 的幂次
    closest_bs = min(powers_of_2, key=lambda x: abs(x - target_bs))
    
    return closest_bs

# 假设 args 包含 train_size
class Args:
    def __init__(self, train_size):
        self.train_size = train_size

args = Args(train_size=100)

# 找到最接近的 batch size
bs = find_closest_power_of_2(args.train_size)

print(f'最佳 batch size 为: {bs}')

最佳 batch size 为: 8


In [8]:
import pickle
with open('./agu/4090.pkl', 'rb') as f:
    print(pickle.load(f))
with open('./agu/3080.pkl', 'rb') as f:
    print(pickle.load(f))
# with open('./agu/1080Ti.pkl', 'rb') as f:
#     print(pickle.load(f))
with open('agu/1080Ti.pkl', 'rb') as f:
    print(pickle.load(f))

[5120, 2.2, 12.0, 256.0]
[8704, 1440, 10.0, 320]
[3584, 1480, 11, 352]


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1x1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=1)
        self.conv3x3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.maxpool3x3 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        
    def forward(self, x):
        # 主路径
        out = self.conv3x3(x)
        
        # 残差连接，1x1 卷积
        residual = self.conv1x1(x)
        
        # 计算残差路径和主路径之和
        out += residual
        
        # 最大池化路径
        out = self.maxpool3x3(out)
        
        return out

# 创建模型实例
model = CustomModel()

# 检查模型结构
print(model)

CustomModel(
  (conv1x1): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
  (conv3x3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool3x3): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
)


In [3]:
pickle.load(open('./agu/core-i7-7820x.pkl', 'rb'))

[4.3, 8, 16, 11.0, 37.5]

In [10]:
df = pickle.load(open('./agu/core-i9-13900k.pkl', 'rb'))
df

[5.2, 16, 32, 30, 59.7]

In [4]:
df = pickle.load(open('./agu/1080Ti.pkl', 'rb'))
df

[3584, 1582, 11, 352]

In [15]:
norm = [7, 32, 32, 40, 60]
for i in range(len(norm)):
    df[i] /= norm[i]
df

[0.10612244897959185, 0.015625, 0.03125, 0.01875, 0.016583333333333332]

In [17]:
import torch 
df = pickle.load(open('./agu/core-i9-13900k.pkl', 'rb'))
norm = [7, 32, 32, 40, 60]
for i in range(len(norm)):
    df[i] /= norm[i]
aug_data = torch.tensor(df).float().unsqueeze(0)

tensor([[0.7429, 0.5000, 1.0000, 0.7500, 0.9950]])

In [21]:
a = './agu/core-i9-13900k.pkl'
a.split('.')[-2]

'/agu/core-i9-13900k'

In [29]:
import pickle 
df = pickle.load(open('datasets/cpu/desktop-cpu-core-i7-7820x-fp32.pickle', 'rb'))
max_value = 0
for key in df.keys():
    max_value = max(max_value, df[key].max())
max_value

0.009513392448425292

In [ ]:
datasets/cpu/desktop-cpu-core-i7-7820x-fp32.pickle

In [4]:
from dgl import graph

def problematic_function():
    g = graph([])
    g.add_nodes(10)
    g.add_edges(list(range(9)), list(range(1, 10)))
    return g

problematic_function()


Graph(num_nodes=10, num_edges=9,
      ndata_schemes={}
      edata_schemes={})

In [5]:
class InferenceNetwork(nn.Module):

    def __init__(self, hw_embed_on, hw_embed_dim, layer_size, determ):
        super(InferenceNetwork, self).__init__()
        self.num_channel = 3
        self.hw_embed_on = hw_embed_on
        self.layer_size = layer_size
        # 是否是确定性推断
        self.determ = determ

        # z 编码器，用于将硬件嵌入转化为潜在空间的均值和标准差
        self.z_encoder = nn.Sequential(*[
            nn.Linear(hw_embed_dim, 10),   # 输入硬件嵌入维度，输出 10 维度
            nn.ReLU(),                     # 激活函数 ReLU
            nn.Linear(10, 2*(self.layer_size*2+1)*3)  # 输出 2*(layer_size*2+1)*3 维度
        ])
        self.softplus = nn.Softplus()
        self.softmax = nn.Softmax(dim=0)

    def get_posterior(self, inputs):
        # 获取后验分布函数，输入包括 (x, y, hw_embed)
        (x, y, hw_embed) = inputs
        # 使用 z 编码器对硬件嵌入进行编码，得到 z 的统计值
        z_stats = self.z_encoder(hw_embed)
        # 从 z_stats 中提取均值，前 (layer_size*2+1)*3 个为均值
        mu_z = z_stats[:(self.layer_size*2+1)*3].squeeze()     
        # 从 z_stats 中提取标准差，后 (layer_size*2+1)*3 个为标准差
        sigma_z = z_stats[(self.layer_size*2+1)*3:].squeeze()  
        # 根据均值和经过 softplus 的标准差，生成正态分布
        q_z = torch.distributions.Normal(mu_z, self.softplus(sigma_z))

        return q_z
    
    def kl_diagnormal_stdnormal(self, p):
        pshape = p.mean.shape
        device = p.mean.device
        q = torch.distributions.Normal(torch.zeros(pshape, device=device), torch.ones(pshape, device=device))
        return torch.distributions.kl.kl_divergence(p, q).to(device)

    def forward(self, inputs):
        # 计算后验分布
        q_z = self.get_posterior(inputs)
        # 计算 KL 散度，使用标准正态分布作为参考分布
        kl_z = torch.sum(self.kl_diagnormal_stdnormal(q_z))
        # 从后验分布中采样，或者直接取均值（如果是确定性推断）
        z = None
        kl = 0.

        kl = kl + kl_z  # 累积 KL 散度
        # 如果不是确定性推断，使用 rsample 从后验分布中采样；否则，使用分布的均值
        z_ = q_z.rsample() if not self.determ else q_z.mean 
        # 从 z_ 中提取权重部分（前 (layer_size*2)*3 个元素）
        zw_ = z_[:(self.layer_size*2)*3].squeeze()     
        # 从 z_ 中提取偏置部分（后面的元素）
        zb_ = z_[(self.layer_size*2)*3:].squeeze()     
        # 将权重和偏置保存在字典 z 中
        z = {'w':zw_, 'b':zb_}
        # 返回采样的权重和偏置，以及 KL 散度
        return z, kl

tensor([[0.5000, 1.0000, 0.0000],
        [0.0000, 0.5000, 1.0000]])

In [2]:
import torch

# 第一个变量 graph
graph = torch.tensor([
    [1., 1., 1., 1., 1., 1., 1., 1., 1.],
    [0., 1., 0., 1., 1., 0., 1., 0., 0.],
    [0., 0., 1., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 1., 0., 0., 0., 1., 0.],
    [0., 0., 0., 0., 1., 0., 0., 1., 0.],
    [0., 0., 0., 0., 0., 1., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 1., 0., 1.],
    [0., 0., 0., 0., 0., 0., 0., 1., 1.],
    [0., 0., 0., 0., 0., 0., 0., 0., 1.]
])

# 第二个变量 features
features = torch.tensor([
    [0, 0, 0, 0, 0, 1],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0]
])

# 检查每一行 features，如果为全 0，则在 graph 的第一行将对应位置的值置为 0
for i in range(features.size(0)):
    if torch.all(features[i] == 0):
        graph[0, i] = 0

# 输出修改后的 graph
print(graph)

tensor([[1., 1., 0., 1., 1., 0., 1., 1., 1.],
        [0., 1., 0., 1., 1., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.]])


In [70]:
def send_email(subject, body, receiver_email="zengyuxiang@hnu.edu.cn"):
    import yagmail
    import pickle
    import os
    # 获得主目录下的邮件授权码
    email_code_address = os.path.expanduser('~') + '/qq_smtp_info.pickle'
    try:
        with open(email_code_address, 'rb') as f:
            all_info = pickle.load(f)
        sender_email = all_info['email']
        sender_password = all_info['password']
    except FileNotFoundError:
        print("非管理员，无法发送邮件")
        return False
    try:
        yag = yagmail.SMTP(user=sender_email, password=sender_password, host='smtp.qq.com')
        yag.send(to=receiver_email, subject=subject, contents=body)
        print("邮件发送成功!")
    except Exception as e:
        print(f"发送邮件时出错: {e}")


In [72]:
for i in range(10):
    send_email('爱宝宝', '爱宝宝', '22rqwang@stu.edu.cn')

邮件发送成功!
邮件发送成功!
邮件发送成功!
邮件发送成功!
邮件发送成功!
邮件发送成功!
邮件发送成功!
邮件发送成功!
邮件发送成功!
邮件发送成功!


In [4]:
import pickle
with open('./datasets/cpu/desktop-cpu-core-i7-7820x-fp32.pickle', 'rb') as f:
    data = pickle.load(f)

In [6]:
data

{(0, 0, 0, 2, 2, 4): 0.0015538465976715089,
 (1, 0, 0, 2, 2, 4): 0.0021101105213165283,
 (2, 0, 0, 2, 2, 4): 0.0027110326290130616,
 (3, 0, 0, 2, 2, 4): 0.004154490232467651,
 (4, 0, 0, 2, 2, 4): 0.004325815439224243,
 (0, 1, 0, 2, 2, 4): 0.0035810256004333496,
 (1, 1, 0, 2, 2, 4): 0.0038075721263885498,
 (2, 1, 0, 2, 2, 4): 0.003969377756118774,
 (3, 1, 0, 2, 2, 4): 0.004825694561004639,
 (4, 1, 0, 2, 2, 4): 0.0044181067943573,
 (0, 2, 0, 2, 2, 4): 0.004203616380691528,
 (1, 2, 0, 2, 2, 4): 0.0044976317882537845,
 (2, 2, 0, 2, 2, 4): 0.004518198966979981,
 (3, 2, 0, 2, 2, 4): 0.005019345283508301,
 (4, 2, 0, 2, 2, 4): 0.00474911093711853,
 (0, 3, 0, 2, 2, 4): 0.005515866279602051,
 (1, 3, 0, 2, 2, 4): 0.005941680669784546,
 (2, 3, 0, 2, 2, 4): 0.0059148168563842776,
 (3, 3, 0, 2, 2, 4): 0.006303913593292236,
 (4, 3, 0, 2, 2, 4): 0.006008708477020263,
 (0, 4, 0, 2, 2, 4): 0.005678647756576538,
 (1, 4, 0, 2, 2, 4): 0.006145888328552246,
 (2, 4, 0, 2, 2, 4): 0.005960286855697632,
 (3, 4,